### Start and Run All

In [2]:
import os
import pandas as pd
from urllib import request
from bs4 import BeautifulSoup
# from datetime import date, timedelta
from datetime import datetime
from time import sleep, strftime
current_time = datetime.now()
current_time

datetime.datetime(2025, 1, 22, 21, 38, 29, 285926)

In [4]:
url = "https://www.set.or.th/en/market/product/stock/quote/"
percent_span = '' 

In [6]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [8]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"OSD path (osd_path): {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
OSD path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [10]:
file_name = "name-ttl.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file : {input_file}") 

Input file : C:\Users\PC1\OneDrive\A5\Data\name-ttl.csv


In [12]:
file_name = "price-hilo.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [14]:
print(f"output_file (output_file): {output_file}") 
print(f"icd_file (icd_file): {icd_file}") 
print(f"god_file (god_file): {god_file}") 
print(f"osd_file (osd_file): {osd_file}")

output_file (output_file): C:\Users\PC1\OneDrive\A5\Data\price-hilo.csv
icd_file (icd_file): C:\Users\PC1\iCloudDrive\Data\price-hilo.csv
god_file (god_file): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\price-hilo.csv
osd_file (osd_file): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\price-hilo.csv


In [16]:
def cvt_str_to_flt(input_str):
    # Check for the special pattern and return 0
    if input_str == '(-)':
        return 0
    
    # Remove parentheses and percent sign
    cleaned_str = input_str.replace('(', '').replace(')', '').replace('%', '')

    # Convert to float
    try:
        return float(cleaned_str)
    except ValueError:
        # Handle the case where the conversion fails
        return None
 

In [18]:
def get_stock_data(in_df):

    in_df.columns = ['name']
    
    output_columns = ['name', 'price', 'today_low', 'today_high', 'year_low', 'year_high', 'percent', 'updated_at']
    # Initialize an empty DataFrame with specified columns
    out_df = pd.DataFrame(columns=output_columns)
    
    for index, row in in_df.iterrows():
        name = str(row['name']).upper()
        response = request.urlopen(url + name + '/price')
        html_page = BeautifulSoup(response, 'html.parser')
        
  #      price = html_page.find('div', class_='value text-white mb-0 me-2 lh-1 stock-info').text.strip()        
  #      if (price == '-'):
  #          price = '0.00'        

        price_div = html_page.find('div', class_='value text-white mb-0 me-2 lh-1 stock-info')
        if price_div:
            price = price_div.text.strip()
        else:
            print(f"Price div not found for {name}")
            price = '0.00'     
            
        # Try to find the <h3> tag with the class 'theme-success'
        h3_tag = html_page.find('h3', class_='theme-success')
        if h3_tag:
        # Find all <span> tags within the found <h3> tag
            span_tags = h3_tag.find_all('span')
    
            if len(span_tags) > 1:
                # Extract the text from the second <span> tag
                percent_span = span_tags[1].text
                # print(percent_span)
                
        # Try to find the <h3> tag with the class 'theme-danger'
        h3_tag = html_page.find('h3', class_='theme-danger')
        if h3_tag:
        # Find all <span> tags within the found <h3> tag
            span_tags = h3_tag.find_all('span')
    
            if len(span_tags) > 1:
                # Extract the text from the second <span> tag
                percent_span = span_tags[1].text
                # print(percent_span)    
                
        # Try to find the <h3> tag with the class 'theme-danger'
        h3_tag = html_page.find('h3', class_='theme-normal')
        if h3_tag:
        # Find all <span> tags within the found <h3> tag
            span_tags = h3_tag.find_all('span')
    
            if len(span_tags) > 1:
                # Extract the text from the second <span> tag
                percent_span = span_tags[1].text
                # print(percent_span)   
                
        percent = cvt_str_to_flt(percent_span)
                
        values_low = html_page.find_all('span', class_='title-font-family fs-16px fw-bolder me-auto lh-1')
        today_low = values_low[0].text
        if (today_low == '-'):
            today_low = '0.00'
        
        values_high = html_page.find_all('span', class_='title-font-family fs-16px fw-bolder lh-1')
        today_high = values_high[0].text
        if (today_high == '-'):
            today_high = '0.00'         
        
        year_low = values_low[1].text        
        year_high = values_high[1].text     
        
        # Find the div with the specific class, then find the span within it
        div = html_page.find('div', class_='price-detail-date d-flex align-items-center text-nowrap fs-12px')
        if div:
            span = div.find('span')
            if span:
                text = span.get_text()
#                 print(text)
            else:
                print("Span not found")
        else:
            print("Div not found")
            
        # Extract the date part from the string
        # Assuming the date is always in the format 'dd Mon yyyy'
        date_str = ' '.join(text.split()[2:5])

        # Parse the date string into a datetime object
        date_obj = datetime.strptime(date_str, '%d %b %Y')

        # Format the datetime object to 'yyyy-mm-dd'
        updated_at = date_obj.strftime('%Y-%m-%d')

#         print(updated_at)
        
        # Append row to the DataFrame and print the row
        out_df.loc[len(out_df)] = [name, price, today_low, today_high, year_low, year_high, percent, updated_at]
#       print(out_df.iloc[-1].to_string(header=False, index=False)) 
        print(','.join(map(str, out_df.iloc[-1])))
        sleep(1)
        
    return out_df  # return the final dataframe with all values populated

In [20]:
start_time = strftime("%I:%M %p")
print('Start at: ', start_time)

Start at:  09:40 PM


In [22]:
df = pd.read_csv(input_file, header=None)
out_df = get_stock_data(df)

ACE,1.26,1.25,1.27,1.25,1.60,0.0,2025-01-22
ADVANC,289.00,286.00,290.00,193.00,299.00,0.35,2025-01-22
AEONTS,118.00,116.50,119.00,107.00,168.50,1.29,2025-01-22
AH,14.20,14.00,15.00,14.30,30.50,-4.7,2025-01-22
AIE,0.79,0.77,0.79,0.74,1.51,0.0,2025-01-22
AIMIRT,10.50,10.40,10.50,10.20,11.20,0.0,2025-01-22
AIT,4.34,4.20,4.36,3.62,5.05,1.88,2025-01-22
AJ,2.44,2.38,2.46,2.36,7.15,0.83,2025-01-22
AMATA,27.50,26.75,27.75,20.10,31.00,2.8,2025-01-22
ANAN,0.51,0.50,0.52,0.48,1.03,0.0,2025-01-22
AOT,58.25,57.75,58.50,55.00,67.50,0.87,2025-01-22
AP,7.80,7.75,7.85,7.50,11.20,0.65,2025-01-22
ASIAN,7.75,7.70,7.80,6.35,11.20,0.0,2025-01-22
ASK,8.85,8.70,8.95,8.40,20.30,2.31,2025-01-22
ASP,2.34,2.34,2.36,2.12,2.88,0.0,2025-01-22
ASW,7.85,7.80,7.85,7.35,8.45,0.64,2025-01-22
AWC,3.10,3.10,3.16,2.88,4.58,0.0,2025-01-22
BA,19.50,19.30,19.80,14.20,26.00,-0.51,2025-01-22
BAM,5.70,5.65,5.85,5.35,10.40,-0.87,2025-01-22
BANPU,5.20,5.20,5.35,4.38,7.50,0.97,2025-01-22
BAY,23.90,23.80,24.10,22.20,29.00,0.0,2025-01

In [24]:
end_time = strftime("%I:%M %p")
print('End at: ', end_time)

End at:  09:49 PM


In [26]:
out_df.query('price == "-"')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at
25,BCT,-,0.00,0.00,53.00,76.75,0.0,2025-01-22


In [28]:
out_df.shape

(221, 8)

In [30]:
out_df = out_df[out_df['name'] != 'BCT']

In [36]:
out_df = out_df[out_df['name'] != 'LPF']

In [38]:
out_df = out_df[out_df['name'] != 'TYCN']

In [40]:
out_df = out_df[out_df['name'] != 'GRAMMY']

In [42]:
out_df = out_df[out_df['name'] != 'KYE']

In [32]:
out_df.shape

(220, 8)

In [34]:
out_df['price'] = out_df['price'].astype('float')
out_df['today_low'] = out_df['today_low'].astype('float')
out_df['today_high'] = out_df['today_high'].astype('float')
out_df['year_low'] = out_df['year_low'].astype('float')
out_df['year_high'] = out_df['year_high'].astype('float')
out_df['percent'] = out_df['percent'].astype('float')

In [36]:
sorted_out_df = out_df.sort_values('percent')

In [38]:
sorted_out_df.query('today_low < year_low')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at
202,TSE,0.69,0.69,0.73,0.72,2.12,-5.48,2025-01-22
3,AH,14.20,14.00,15.00,14.30,30.50,-4.70,2025-01-22
122,NOBLE,2.42,2.42,2.50,2.46,4.10,-3.20,2025-01-22
97,KEX,1.40,1.39,1.42,1.40,6.50,-2.10,2025-01-22
214,VNG,2.94,2.90,2.94,2.92,3.92,-1.34,2025-01-22
66,FPT,9.20,9.10,9.30,9.30,15.40,-1.08,2025-01-22


In [40]:
out_df.query('today_low < year_low').shape[0]

6

In [42]:
sorted_out_df.query('today_high > year_high')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at
154,SCB,124.5,122.0,125.0,100.0,123.0,1.63,2025-01-22
101,KTB,23.0,21.8,23.0,15.4,22.4,5.50,2025-01-22


In [44]:
out_df.query('today_high > year_high').shape[0]

2

In [46]:
file_name = "Price-Hilo.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [48]:
print(f"output_file (output_file): {output_file}") 
print(f"icd_file (icd_file): {icd_file}") 
print(f"god_file (god_file): {god_file}") 
print(f"osd_file (osd_file): {osd_file}") 

output_file (output_file): C:\Users\PC1\OneDrive\A5\Data\Price-Hilo.csv
icd_file (icd_file): C:\Users\PC1\iCloudDrive\Data\Price-Hilo.csv
god_file (god_file): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\Price-Hilo.csv
osd_file (osd_file): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\Price-Hilo.csv


In [50]:
out_df.to_csv(output_file, index=False)  
out_df.to_csv(god_file, index=False) 
out_df.to_csv(icd_file, index=False)  
out_df.to_csv(osd_file, index=False) 

In [52]:
current_time

datetime.datetime(2025, 1, 22, 21, 38, 29, 285926)